# Baseline Analysis of Validation Data

In [1]:
import os
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', 500)

# For Visualization
import matplotlib.pyplot as plt
import seaborn as sns
#%config InlineBackend.figure_format = 'svg'

os.chdir('..') #change to parent directory

from pathlib import Path  #for Windows/Linux compatibility
DATAPATH = Path(r'data')


In [2]:

validation = pd.read_csv(DATAPATH / "training" / "validation_data.csv")
training = pd.read_csv(DATAPATH / "training" / "training_data.csv")

validation.head()

,game_id,season,sub_season_id,is_playoff,is_overtime,min,month,day_of_week_sin,day_of_week_cos,month_sin,month_cos,season_progress,h_team_id,h_is_home_team,h_is_win,h_ast_rolling_avg_3_all,h_reb_pct_rolling_avg_3_all,h_fta_rate_rolling_avg_3_all,h_fgm_pct_ast_rolling_avg_3_all,h_pct_pts_2pt_rolling_avg_3_all,h_pct_pts_pitp_rolling_avg_3_all,h_fg_pct_rolling_avg_3_all,h_pct_pts_2pt_mr_rolling_avg_3_all,h_opp_efg_pct_rolling_avg_3_all,h_ts_pct_rolling_avg_3_all,h_opp_pitp_rolling_avg_3_all,h_ast_pct_rolling_avg_3_all,h_oreb_rolling_avg_3_all,h_efg_pct_rolling_avg_3_all,h_dreb_rolling_avg_3_all,h_pct_pts_3pt_rolling_avg_3_all,h_off_rtg_rolling_avg_3_all,h_opp_oreb_pct_rolling_avg_3_all,h_2fgm_pct_uast_rolling_avg_3_all,h_2fgm_pct_ast_rolling_avg_3_all,h_reb_rolling_avg_3_all,h_dreb_pct_rolling_avg_3_all,h_pts_off_to_rolling_avg_3_all,h_pts_rolling_avg_3_all,h_blk_rolling_avg_3_all,h_pct_pts_ft_rolling_avg_3_all,h_tov_pct_rolling_avg_3_all,h_ftm_rolling_avg_3_all,h_net_rtg_rolling_avg_3_all,h_oreb_pct_rolling_avg_3_all,h_opp_pts_off_to_rolling_avg_3_all,h_3pa_rolling_avg_3_all,h_pie_rolling_avg_3_all,h_opp_tov_pct_rolling_avg_3_all,h_3fgm_pct_uast_rolling_avg_3_all,h_fbps_rolling_avg_3_all,h_is_win_rolling_avg_3_all,h_fta_rolling_avg_3_all,h_opp_2nd_pts_rolling_avg_3_all,h_fgm_pct_uast_rolling_avg_3_all,h_tov_rolling_avg_3_all,h_plus_minus_rolling_avg_3_all,h_stl_rolling_avg_3_all,h_2nd_pts_rolling_avg_3_all,h_pf_rolling_avg_3_all,h_ft_pct_rolling_avg_3_all,h_opp_fta_rate_rolling_avg_3_all,h_ast_ratio_rolling_avg_3_all,h_ast_turnover_ratio_rolling_avg_3_all,h_def_rtg_rolling_avg_3_all,h_3pm_rolling_avg_3_all,h_pct_pts_off_to_rolling_avg_3_all,h_pitp_rolling_avg_3_all,h_3fgm_pct_ast_rolling_avg_3_all,h_pace_rolling_avg_3_all,h_pct_fga_2pt_rolling_avg_3_all,h_fga_rolling_avg_3_all,h_fgm_rolling_avg_3_all,h_3p_pct_rolling_avg_3_all,h_opp_fbps_rolling_avg_3_all,h_pct_pts_fbps_rolling_avg_3_all,h_pct_fga_3pt_rolling_avg_3_all,h_ast_rolling_avg_5_all,h_reb_pct_rolling_avg_5_all,h_fta_rate_rolling_avg_5_all,h_fgm_pct_ast_rolling_avg_5_all,h_pct_pts_2pt_rolling_avg_5_all,h_pct_pts_pitp_rolling_avg_5_all,h_fg_pct_rolling_avg_5_all,h_pct_pts_2pt_mr_rolling_avg_5_all,h_opp_efg_pct_rolling_avg_5_all,h_ts_pct_rolling_avg_5_all,h_opp_pitp_rolling_avg_5_all,h_ast_pct_rolling_avg_5_all,h_oreb_rolling_avg_5_all,h_efg_pct_rolling_avg_5_all,h_dreb_rolling_avg_5_all,h_pct_pts_3pt_rolling_avg_5_all,h_off_rtg_rolling_avg_5_all,h_opp_oreb_pct_rolling_avg_5_all,h_2fgm_pct_uast_rolling_avg_5_all,h_2fgm_pct_ast_rolling_avg_5_all,h_reb_rolling_avg_5_all,h_dreb_pct_rolling_avg_5_all,h_pts_off_to_rolling_avg_5_all,h_pts_rolling_avg_5_all,h_blk_rolling_avg_5_all,h_pct_pts_ft_rolling_avg_5_all,h_tov_pct_rolling_avg_5_all,h_ftm_rolling_avg_5_all,h_net_rtg_rolling_avg_5_all,h_oreb_pct_rolling_avg_5_all,h_opp_pts_off_to_rolling_avg_5_all,h_3pa_rolling_avg_5_all,h_pie_rolling_avg_5_all,h_opp_tov_pct_rolling_avg_5_all,h_3fgm_pct_uast_rolling_avg_5_all,h_fbps_rolling_avg_5_all,h_is_win_rolling_avg_5_all,h_fta_rolling_avg_5_all,h_opp_2nd_pts_rolling_avg_5_all,h_fgm_pct_uast_rolling_avg_5_all,h_tov_rolling_avg_5_all,h_plus_minus_rolling_avg_5_all,h_stl_rolling_avg_5_all,h_2nd_pts_rolling_avg_5_all,h_pf_rolling_avg_5_all,h_ft_pct_rolling_avg_5_all,h_opp_fta_rate_rolling_avg_5_all,h_ast_ratio_rolling_avg_5_all,h_ast_turnover_ratio_rolling_avg_5_all,h_def_rtg_rolling_avg_5_all,h_3pm_rolling_avg_5_all,h_pct_pts_off_to_rolling_avg_5_all,h_pitp_rolling_avg_5_all,h_3fgm_pct_ast_rolling_avg_5_all,h_pace_rolling_avg_5_all,h_pct_fga_2pt_rolling_avg_5_all,h_fga_rolling_avg_5_all,h_fgm_rolling_avg_5_all,h_3p_pct_rolling_avg_5_all,h_opp_fbps_rolling_avg_5_all,h_pct_pts_fbps_rolling_avg_5_all,h_pct_fga_3pt_rolling_avg_5_all,h_ast_rolling_avg_7_all,h_reb_pct_rolling_avg_7_all,h_fta_rate_rolling_avg_7_all,h_fgm_pct_ast_rolling_avg_7_all,h_pct_pts_2pt_rolling_avg_7_all,h_pct_pts_pitp_rolling_avg_7_all,h_fg_pct_rolling_avg_7_all,h_pct_pts_2pt_mr_rolling_avg_7_all,h_opp_efg_pct_rolling_a

In [3]:
validation.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 954 entries, 0 to 953
Columns: 2263 entries, game_id to v_elo_win_prob
dtypes: float64(2247), int64(16)
memory usage: 16.5 MB


In [4]:
# predict winner if home team
accuracy = (validation['h_is_win'] == validation['h_is_home_team']).mean()
print(f"Prediction accuracy: {accuracy:.2%}")


Prediction accuracy: 52.52%


In [5]:
# Predict winner if home team's rolling average is greater than visitor's
validation['prediction'] = (validation['h_is_win_rolling_avg_20_all'] > validation['v_is_win_rolling_avg_20_all']).astype(int)

# Calculate accuracy
accuracy = (validation['prediction'] == validation['h_is_win']).mean()

print(f"Prediction accuracy: {accuracy:.2%}")

Prediction accuracy: 60.17%


In [6]:
# Predict winner if elo score is greater
validation['prediction'] = (validation['h_elo_team_before'] > validation['v_elo_team_before']).astype(int)
accuracy = (validation['prediction'] == validation['h_is_win']).mean()
print(f"Prediction accuracy: {accuracy:.2%}")

# find the features most correlated with h_is_win

Prediction accuracy: 60.69%


In [7]:
# predict winner if elo_win_prob is > 0.5
validation['prediction'] = (validation['h_elo_win_prob'] > 0.5).astype(int)
accuracy = (validation['prediction'] == validation['h_is_win']).mean()
print(f"Prediction accuracy: {accuracy:.2%}")


Prediction accuracy: 59.43%


In [8]:
# find the features most correlated with h_is_win
corr_matrix = training.corr()
corr_matrix['h_is_win'].sort_values(ascending=False)


h_is_win                              1.000000
h_elo_win_prob                        0.352174
h_elo_team_before                     0.268602
v_elo_opp_before                      0.268602
h_plus_minus_rolling_avg_20_all       0.260047
                                        ...   
v_3p_pct_rolling_avg_20_home               NaN
v_opp_fbps_rolling_avg_20_home             NaN
v_pct_pts_fbps_rolling_avg_20_home         NaN
v_pct_fga_3pt_rolling_avg_20_home          NaN
v_win_streak_home                          NaN
Name: h_is_win, Length: 2263, dtype: float64

In [9]:
# using training data to find best threshold for:
#  h_elo_win_prob,
#  h_plus_minus_rolling_avg_20_all,

def calculate_accuracy(df, column, threshold):
    predictions = (df[column] > threshold).astype(int)
    accuracy = (predictions == df['h_is_win']).mean()
    return accuracy

def find_best_threshold(df, column, step=0.01):
    min_val = df[column].min()
    max_val = df[column].max()
    thresholds = np.arange(min_val, max_val, step)
    
    best_threshold = min_val
    best_accuracy = 0
    
    for threshold in thresholds:
        accuracy = calculate_accuracy(df, column, threshold)
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_threshold = threshold
    
    return best_threshold, best_accuracy


df = training.copy()

# limit in the training data and see if that improves the accuracy
df = df[df['season'] >= 2019]   

# Find best threshold for h_elo_win_prob
h_elo_win_prob_threshold, h_elo_win_prob_accuracy = find_best_threshold(df, 'h_elo_win_prob')
print(f"Best threshold for h_elo_win_prob: {h_elo_win_prob_threshold:.4f}")
print(f"Accuracy at this threshold: {h_elo_win_prob_accuracy:.4f}")

# Find best threshold for h_plus_minus_rolling_avg_20_all
h_plus_minus_rolling_avg_20_all_threshold, h_plus_minus_rolling_avg_20_all_accuracy = find_best_threshold(df, 'h_plus_minus_rolling_avg_20_all')
print(f"Best threshold for h_plus_minus_rolling_avg_20_all: {h_plus_minus_rolling_avg_20_all_threshold:.4f}")
print(f"Accuracy at this threshold: {h_plus_minus_rolling_avg_20_all_accuracy:.4f}")



Best threshold for h_elo_win_prob: 0.5254
Accuracy at this threshold: 0.6410
Best threshold for h_plus_minus_rolling_avg_20_all: -1.9914
Accuracy at this threshold: 0.6256


In [10]:
# now use the validation data to calculate the accuracy of the predictions using the best threshold for each feature

# h_elo_win_prob
validation['prediction'] = (validation['h_elo_win_prob'] > h_elo_win_prob_threshold).astype(int)
accuracy = (validation['prediction'] == validation['h_is_win']).mean()
print(f"Accuracy for h_elo_win_prob: {accuracy:.2%}")

# h_plus_minus_rolling_avg_20_all
validation['prediction'] = (validation['h_plus_minus_rolling_avg_20_all'] > h_plus_minus_rolling_avg_20_all_threshold).astype(int)
accuracy = (validation['prediction'] == validation['h_is_win']).mean()
print(f"Accuracy for h_plus_minus_rolling_avg_20_all: {accuracy:.2%}")



Accuracy for h_elo_win_prob: 59.33%
Accuracy for h_plus_minus_rolling_avg_20_all: 62.37%
